In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,032 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,374 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy Release.g

In [ ]:
!apt-get install nvidia-cuda-toolkit g++

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
g++ is already the newest version (4:11.2.0-1ubuntu1).
g++ set to manually installed.
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libaccinj64-11.5 libatk-wrapper-java libatk-wrapper-java-jni
  libbabeltrace1 libcub-dev libcublas11 libcublaslt11 libcudart11.0 libcufft10 libcufftw10
  libcuinj64-11.5 libcupti-dev libcupti-doc libcupti11.5 libcurand10 libcusolver11 libcusolvermg11
  libcusparse11 libdebuginfod-common libdebuginfod1 libegl-dev libfontenc1 libgail-common libgail18
  libgl-dev libgl1-mesa-dev libgles-dev libgles1 libglvnd-core-dev libglvnd-dev libglx-dev
  libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libipt2 libnppc11 libnppial11 libnppicc11 libnppidei11
  libnppif11 libnppig11 libnppim11 libnppist11 libnppisu11 libnppitc11 libnpps11 libnvblas11
  libnvidia-compute-495 libnvidia-compute-510 libnvidia-compute-535 libnvidia-ml-dev libnv

In [ ]:
%%writefile matrix_mul.cu
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include <cuda_runtime.h>
#include <omp.h>

#define BLOCK_SIZE 16
#define PRINT_LIMIT 10 // Limit to print elements of large matrices

// Function to print a matrix (with limits for large matrices)
void print_matrix(int* matrix, int rows, int cols, const char* name)
{
    printf("Matrix %s (%d x %d):\n", name, rows, cols);
    for (int i = 0; i < rows && i < PRINT_LIMIT; ++i)
    {
        for (int j = 0; j < cols && j < PRINT_LIMIT; ++j)
        {
            printf("%4d ", matrix[i * cols + j]);
        }
        if (cols > PRINT_LIMIT)
        {
            printf("... "); // Print ellipsis if there are more columns
        }
        printf("\n");
    }
    if (rows > PRINT_LIMIT)
    {
        printf("... \n"); // Print ellipsis if there are more rows
    }
    printf("\n");
}

// CUDA kernel for general matrix multiplication
__global__ void gpu_matrix_mult(int* a, int* b, int* c, int m, int n, int k)
{
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int sum = 0;
    if (col < k && row < m)
    {
        for (int i = 0; i < n; i++)
        {
            sum += a[row * n + i] * b[i * k + col];
        }
        c[row * k + col] = sum;
    }
}

// CUDA kernel for square matrix multiplication
__global__ void gpu_square_matrix_mult(int* d_a, int* d_b, int* d_result, int n)
{
    __shared__ int tile_a[BLOCK_SIZE][BLOCK_SIZE];
    __shared__ int tile_b[BLOCK_SIZE][BLOCK_SIZE];

    int row = blockIdx.y * BLOCK_SIZE + threadIdx.y;
    int col = blockIdx.x * BLOCK_SIZE + threadIdx.x;
    int tmp = 0;
    int idx;

    for (int sub = 0; sub < gridDim.x; ++sub)
    {
        idx = row * n + sub * BLOCK_SIZE + threadIdx.x;
        tile_a[threadIdx.y][threadIdx.x] = (idx < n * n) ? d_a[idx] : 0;

        idx = (sub * BLOCK_SIZE + threadIdx.y) * n + col;
        tile_b[threadIdx.y][threadIdx.x] = (idx < n * n) ? d_b[idx] : 0;

        __syncthreads();

        for (int k = 0; k < BLOCK_SIZE; ++k)
        {
            tmp += tile_a[threadIdx.y][k] * tile_b[k][threadIdx.x];
        }
        __syncthreads();
    }

    if (row < n && col < n)
    {
        d_result[row * n + col] = tmp;
    }
}

// OpenMP function for matrix multiplication (parallelized)
void openmp_matrix_mult(int* h_a, int* h_b, int* h_c, int m, int n, int k)
{
#pragma omp parallel for collapse(2)
    for (int i = 0; i < m; ++i)
    {
        for (int j = 0; j < k; ++j)
        {
            int tmp = 0;
            for (int h = 0; h < n; ++h)
            {
                tmp += h_a[i * n + h] * h_b[h * k + j];
            }
            h_c[i * k + j] = tmp;
        }
    }
}

// Normal (sequential) matrix multiplication function
void cpu_matrix_mult(int* h_a, int* h_b, int* h_result, int m, int n, int k)
{
    for (int i = 0; i < m; ++i)
    {
        for (int j = 0; j < k; ++j)
        {
            int tmp = 0;
            for (int h = 0; h < n; ++h)
            {
                tmp += h_a[i * n + h] * h_b[h * k + j];
            }
            h_result[i * k + j] = tmp;
        }
    }
}

// Main function
int main(int argc, char const* argv[])
{
    int m, n, k;
    srand(3333); // Fixed seed
    printf("Please type in m, n, and k: ");
    scanf("%d %d %d", &m, &n, &k);

    // Allocate memory in host RAM
    int* h_a, * h_b, * h_c, * h_cc;
    cudaMallocHost((void**)&h_a, sizeof(int) * m * n);
    cudaMallocHost((void**)&h_b, sizeof(int) * n * k);
    cudaMallocHost((void**)&h_c, sizeof(int) * m * k);
    cudaMallocHost((void**)&h_cc, sizeof(int) * m * k);

    // Random initialize matrix A
    for (int i = 0; i < m; ++i)
    {
        for (int j = 0; j < n; ++j)
        {
            h_a[i * n + j] = rand() % 1024;
        }
    }

    // Random initialize matrix B
    for (int i = 0; i < n; ++i)
    {
        for (int j = 0; j < k; ++j)
        {
            h_b[i * k + j] = rand() % 1024;
        }
    }

    // Print matrices A and B
    print_matrix(h_a, m, n, "A");
    print_matrix(h_b, n, k, "B");

    float gpu_elapsed_time_ms, cpu_elapsed_time_ms, normal_elapsed_time_ms;

    // Start measuring GPU execution time
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start, 0);

    // Allocate memory space on the device
    int* d_a, * d_b, * d_c;
    cudaMalloc((void**)&d_a, sizeof(int) * m * n);
    cudaMalloc((void**)&d_b, sizeof(int) * n * k);
    cudaMalloc((void**)&d_c, sizeof(int) * m * k);

    // Copy matrix A and B from host to device memory
    cudaMemcpy(d_a, h_a, sizeof(int) * m * n, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, sizeof(int) * n * k, cudaMemcpyHostToDevice);

    unsigned int grid_rows = (m + BLOCK_SIZE - 1) / BLOCK_SIZE;
    unsigned int grid_cols = (k + BLOCK_SIZE - 1) / BLOCK_SIZE;
    dim3 dimGrid(grid_cols, grid_rows);
    dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE);

    // Launch the appropriate kernel
    if (m == n && n == k)
    {
        gpu_square_matrix_mult << <dimGrid, dimBlock >> > (d_a, d_b, d_c, n);
    }
    else
    {
        gpu_matrix_mult << <dimGrid, dimBlock >> > (d_a, d_b, d_c, m, n, k);
    }

    // Transfer results from device to host
    cudaMemcpy(h_c, d_c, sizeof(int) * m * k, cudaMemcpyDeviceToHost);
    cudaDeviceSynchronize(); // Wait for GPU to finish
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    // Compute time elapsed on GPU computing
    cudaEventElapsedTime(&gpu_elapsed_time_ms, start, stop);
    printf("Time elapsed on matrix multiplication of %dx%d . %dx%d on GPU: %f ms.\n", m, n, n, k, gpu_elapsed_time_ms);

    // Print result matrix C (GPU result)
    print_matrix(h_c, m, k, "C (GPU Result)");

    // Start measuring normal (sequential) execution time
    double start_time = omp_get_wtime();
    cpu_matrix_mult(h_a, h_b, h_cc, m, n, k);
    double end_time = omp_get_wtime();
    normal_elapsed_time_ms = (end_time - start_time) * 1000.0; // Convert to milliseconds
    printf("Time elapsed on normal matrix multiplication of %dx%d . %dx%d on CPU: %f ms.\n", m, n, n, k, normal_elapsed_time_ms);

    // Print result matrix C (CPU result)
    print_matrix(h_cc, m, k, "C (CPU Result)");

    // Start measuring CPU execution time using OpenMP
    start_time = omp_get_wtime();
    openmp_matrix_mult(h_a, h_b, h_cc, m, n, k);
    end_time = omp_get_wtime();
    cpu_elapsed_time_ms = (end_time - start_time) * 1000.0; // Convert to milliseconds
    printf("Time elapsed on matrix multiplication of %dx%d . %dx%d on CPU (OpenMP): %f ms.\n", m, n, n, k, cpu_elapsed_time_ms);

    // Compare the results
    int all_ok = 1;
    for (int i = 0; i < m; i++)
    {
        for (int j = 0; j < k; j++)
        {
            if (h_cc[i * k + j] != h_c[i * k + j])
            {
                all_ok = 0;
                printf("Mismatch at [%d][%d]: GPU=%d, CPU=%d\n", i, j, h_c[i * k + j], h_cc[i * k + j]);
                break;
            }
        }
        if (!all_ok) break;
    }

    printf("Matrix multiplication %s\n", all_ok ? "successful!" : "failed.");

    // Free GPU memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    // Free CPU memory
    cudaFreeHost(h_a);
    cudaFreeHost(h_b);
    cudaFreeHost(h_c);
    cudaFreeHost(h_cc);

    return 0;
}


Overwriting matrix_mul.cu


In [ ]:
!nvcc -o matrix_mul matrix_mul.cu -Xcompiler -fopenmp

In [ ]:
!./matrix_mul

Please type in m, n, and k: 3 3 3
Matrix A (3 x 3):
 931  834  940 
 850  189  662 
 830  362  338 

Matrix B (3 x 3):
 581  406  650 
 585  581   93 
1020  522  689 

Time elapsed on matrix multiplication of 3x3 . 3x3 on GPU: 0.461952 ms.
Matrix C (GPU Result) (3 x 3):
1987601 1353220 1330372 
1279655 800473 1026195 
1038760 723738 806048 

Time elapsed on normal matrix multiplication of 3x3 . 3x3 on CPU: 0.000850 ms.
Matrix C (CPU Result) (3 x 3):
1987601 1353220 1330372 
1279655 800473 1026195 
1038760 723738 806048 

Time elapsed on matrix multiplication of 3x3 . 3x3 on CPU (OpenMP): 0.070515 ms.
Matrix multiplication successful!


In [ ]:
!./matrix_mul

Please type in m, n, and k: 10 10 10
Matrix A (10 x 10):
 931  834  940  850  189  662  830  362  338  581 
 406  650  585  581   93 1020  522  689  446  372 
 855  689  468  752  749  626  607  216  241  951 
 341  148  762  258  998  951  920  804  290  234 
 362  696  884  947  254  977  943  776  643  365 
 124  474   30  592  202  779  194  809  995  436 
 736  313  584  474  571  559  402  467  339  692 
 701  701  364  561  624  618  514  543  370  133 
 908  495  607  939   63  809  694  258  594  666 
 694  307  979  254  781  526  813  159  993  129 

Matrix B (10 x 10):
 851  670  830  192  207  431  810  721  974  157 
 855  859  652  438  774  715  224  444  973  818 
  86  643  101   41  898  883  567  687   18  536 
 816  870  182  623   38  389   30  848   87 1004 
1005  942  839  633  356  589  325  580   10  274 
 375   96  918  476  138  792  335  705  455  354 
 218  248  200  400  871  238  790  901   62  877 
 881   44  795  697  677  127  262 1002  708  272 
 253

In [ ]:
!./matrix_mul

Please type in m, n, and k: 100 100 100
Matrix A (100 x 100):
 931  834  940  850  189  662  830  362  338  581 ... 
 851  670  830  192  207  431  810  721  974  157 ... 
 426  200  745  197  897  399  324  135  377    8 ... 
 693  528  188  569  161  106  232 1020  814  966 ... 
 420  457  319  163  868   50   92  782  724  731 ... 
 757  417  832   12  185  610  178  313  170   90 ... 
 201  934  996  618  329  269  552  656  873  129 ... 
 766  407  197  916  553  944  195  794  228  132 ... 
 879  637  824  227  891  862   87  369  838  463 ... 
  79  327  299  163  745  254  238  518  178  469 ... 
... 

Matrix B (100 x 100):
 452 1014  939  577  256  535  168  298  563  890 ... 
 461  349  166  770   52  133  600  506  362  117 ... 
 206  269  636  784  528    0  604  403  785  667 ... 
 424 1003  920  778  663  551   19  162  464  492 ... 
 123   73  920   23  946  951  513  944  533  381 ... 
 318  854  255  790  407  835  777  529  236  913 ... 
 369  270  561   76  520  910 

In [ ]:
!./matrix_mul


Please type in m, n, and k: 1000 1000 1000
Matrix A (1000 x 1000):
 931  834  940  850  189  662  830  362  338  581 ... 
 776  900  474  992  135   83  466  246  416  122 ... 
 921  740  562 1007  438  160  755  179  291  881 ... 
 448  332   24  378  251  638  644  648  650  922 ... 
 675  690   22  899  326  230  293  265  607  937 ... 
 143  938   74  596  491  794  813  205  818  964 ... 
 534  224 1007   42  536  315  895   25   89  278 ... 
 831  542  837  457  959  117   12  360  362  269 ... 
 142    3 1010  677  646  616  899  284  317  166 ... 
 980  307 1017  494  180  639  287   43  579  992 ... 
... 

Matrix B (1000 x 1000):
 403  455  544  980  154 1006  441  342  147  579 ... 
 357   26  366  776  984  670  288  208  165  456 ... 
 423  629  730  275  865  443  287  716   18  106 ... 
 140  231  732  551  160   94  669  150  777  631 ... 
 918  544  771  606   99  906  399  390  969   96 ... 
 943  607  810  634  328   97  697  839  937  601 ... 
 402  364  119  948  13

In [ ]:
!./matrix_mul


Please type in m, n, and k: 5000 5000 5000
Matrix A (5000 x 5000):
 931  834  940  850  189  662  830  362  338  581 ... 
 143  938   74  596  491  794  813  205  818  964 ... 
 452 1014  939  577  256  535  168  298  563  890 ... 
  63  847  302  530  284  705  296  406  613  802 ... 
  39  164  527  402  867  133  230  884  784  555 ... 
 884   45  507  914  245  475  239  977  519  932 ... 
 935  424  386   44  928  214  247  322  771  138 ... 
  71  721  575  820  277  774  389  280  301  403 ... 
 377  375   81  969  478   80   65  609  139  118 ... 
  40  624   67  992  633   27  188  619  676  314 ... 
... 

Matrix B (5000 x 5000):
 312  964  864   84  325  818  974  284  113   91 ... 
 376  903  758  826  249  219  158  176  230  419 ... 
 126  657  996  957  835  284  247  872   51  469 ... 
 944  227  821  254   84  689  455  773  391  426 ... 
 369  584  309  767  693  797  199  290  955  708 ... 
 538  883  524  137  875  954   71   90  885  273 ... 
 880  121  744  771  49